I read in the imported file from Converting 

In [16]:
use "..\input_data\NYPD_Motor_Vehicle_Collisions.dta",clear
notes list 
//Check that notes is "Downloaded 28 Sep 2018 15:48:00"




_dta:
  1.  "Downloaded 28 Sep 2018 15:48:00"


In [17]:
// Formating
order(uniquekey), before(date)
gen date2 = date(date, "MDY"),after(date)
format date2 %td
drop date
rename date2 date

gen time2 = clock(time, "hm"),after(time)
format time2 %tcHH:MM
drop time
rename time2 time

gen latenight = (time > tc(23:00:00) & time <= tc(23:59:59)) | (time >= tc(00:00:00) & time < tc(05:00:00)), after(time)
tab latenight

rename numberofpersonsinjured personsinjured
rename numberofpersonskilled personskilled
rename numberofpedestriansinjured pedestriansinjured
rename numberofpedestrianskilled pedestrianskilled
rename numberofcyclistinjured cyclistinjured
rename numberofcyclistkilled cyclistkilled
rename numberofmotoristinjured motoristinjured
rename numberofmotoristkilled motoristkilled



(1 missing value generated)




(1 missing value generated)






  latenight |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |  1,218,191       90.16       90.16
          1 |    133,024        9.84      100.00
------------+-----------------------------------
      Total |  1,351,215      100.00










In [18]:
// Fill in the missing Longditude and Latitude points

gen year = year(date), before(date)
gen mi_latlong = mi(latitude) & mi(longitude), after(longitude)
tab mi_latlong year, column

duplicates tag onstreetname crossstreetname if ///
!mi(onstreetname) & !mi(crossstreetname), gen(dup)
tab dup

replace onstreetname = strtrim(onstreetname)
replace crossstreetname = strtrim(crossstreetname)
sort onstreetname crossstreetname borough zipcode


(1 missing value generated)



+-------------------+
| Key               |
|-------------------|
|     frequency     |
| column percentage |
+-------------------+

           |                                     year
mi_latlong |      2012       2013       2014       2015       2016       2017       2018 |     Total
-----------+-----------------------------------------------------------------------------+----------
         0 |    85,450    171,914    172,724    182,956    145,733    212,986    155,901 | 1,127,664 
           |     84.99      84.39      83.84      84.05      63.96      92.84      93.93 |     83.46 
-----------+-----------------------------------------------------------------------------+----------
         1 |    15,089     31,809     33,302     34,731     82,104     16,437     10,078 |   223,550 
           |     15.01      15.61      16.16      15.95      36.04       7.16       6.07 |     16.54 
-----------+----------------------------------------------------------

        139 |        840        0.09       94.20
        140 |        705        0.07       94.27
        141 |      1,136        0.12       94.38
        142 |        286        0.03       94.41
        144 |        725        0.07       94.49
        145 |      1,022        0.10       94.59
        146 |        588        0.06       94.65
        147 |        444        0.05       94.70
        148 |        447        0.05       94.74
        150 |      1,057        0.11       94.85
        151 |        760        0.08       94.93
        152 |        153        0.02       94.94
        153 |        154        0.02       94.96
        154 |        310        0.03       94.99
        155 |        312        0.03       95.02
        156 |        157        0.02       95.04
        157 |        316        0.03       95.07
        158 |        636        0.06       95.14
        159 |        480        0.05       95.18
        160 |        966        0.10       95.28
        161 |       

In [19]:
// If borough, zip, on street and cross street are the same 
// we use the latitude & longitude for the missing

replace latitude = latitude[_n-1] if ///
mi(latitude) & ///
!mi(latitude[_n-1]) & ///
!mi(borough) & ///
!mi(borough[_n-1]) & ///
!mi(zipcode) & ///
!mi(zipcode[_n-1]) & ///
!mi(onstreetname) & ///
!mi(onstreetname[_n-1]) & ///
!mi(crossstreetname) & ///
!mi(crossstreetname[_n-1]) & ///
(borough == borough[_n-1]) & ///
(zipcode == zipcode[_n-1]) & ///
(onstreetname == onstreetname[_n-1]) & ///
(crossstreetname ==  crossstreetname[_n-1])

replace longitude = longitude[_n-1] if ///
mi(longitude) & ///
!mi(longitude[_n-1]) & ///
!mi(borough) & ///
!mi(borough[_n-1]) & ///
!mi(zipcode) & ///
!mi(zipcode[_n-1]) & ///
!mi(onstreetname) & ///
!mi(onstreetname[_n-1]) & ///
!mi(crossstreetname) & ///
!mi(crossstreetname[_n-1]) & ///
(borough == borough[_n-1]) & ///
(zipcode == zipcode[_n-1]) & ///
(onstreetname == onstreetname[_n-1]) & ///
(crossstreetname ==  crossstreetname[_n-1])




(20,720 real changes made)

(20,720 real changes made)


imput_latlong indicates that the coordinates for this observation was inputed

In [20]:
gen imput_latlong = (mi_latlong ==1 & !mi(latitude) & !mi(longitude)), after(mi_latlong)
tab imput_latlong

observations with missing are removed

In [21]:
gen mi_latlong2 = mi(latitude) & mi(longitude), after(imput_latlong)
tab mi_latlong2 year, column

// Drop observations which are still missing
drop if mi_latlong2 == 1




+-------------------+
| Key               |
|-------------------|
|     frequency     |
| column percentage |
+-------------------+

mi_latlong |                                     year
         2 |      2012       2013       2014       2015       2016       2017       2018 |     Total
-----------+-----------------------------------------------------------------------------+----------
         0 |    85,452    171,917    172,730    182,958    162,760    215,096    157,471 | 1,148,384 
           |     84.99      84.39      83.84      84.05      71.44      93.76      94.87 |     84.99 
-----------+-----------------------------------------------------------------------------+----------
         1 |    15,087     31,806     33,296     34,729     65,077     14,327      8,508 |   202,830 
           |     15.01      15.61      16.16      15.95      28.56       6.24       5.13 |     15.01 
-----------+-----------------------------------------------------------------------------+---------

observation with nonsensical coordinates are removed

In [22]:
drop if longitude < -74.5 | longitude > -73
drop if latitude < 40 |  latitude > 41


(458 observations deleted)

(4 observations deleted)


In [ ]:
// Mapping
twoway scatter latitude longitude, msymbol(o) msize(tiny)
graph export